In [ ]:
import os
import SimpleITK as sitk
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider


def adjust_images_to_reference_img(
    image_paths, verbose=True, visualise=False, 
    reference_image_path="/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/nnunet/nnUNet_raw/Dataset001_brats/imagesTr/BRATS_0000_0000.nii"
):
    """
    Adjust all images in the list to match the reference image.

    Args:
        image_paths (list): A list of file paths to the images (NIfTI format) to be adjusted.
        verbose (bool): Whether to print information about the images.
        visualise (bool): Whether to visualize the images before and after resampling.
        reference_image_path (str): Path to the reference image.

    Returns:
        list: A list of resampled NIfTI images.
    """
    # Load the reference image
    reference_img_nib = nib.load(reference_image_path)
    reference_image_sitk = sitk.GetImageFromArray(reference_img_nib.get_fdata())
    reference_image_sitk.SetOrigin(reference_img_nib.affine[:3, 3])
    reference_image_sitk.SetDirection(reference_img_nib.affine[:3, :3].flatten())

    resampled_images = []

    for image_path in image_paths:
        # Load the image
        img_nib = nib.load(image_path)
        img_sitk = sitk.GetImageFromArray(img_nib.get_fdata())
        img_sitk.SetOrigin(img_nib.affine[:3, 3])
        img_sitk.SetDirection(img_nib.affine[:3, :3].flatten())

        # Compute initial transform
        initial_transform = sitk.CenteredTransformInitializer(
            reference_image_sitk, img_sitk, sitk.Euler3DTransform(), sitk.CenteredTransformInitializerFilter.GEOMETRY)
        
        # Resample the image
        resampled_img_sitk = sitk.Resample(
            img_sitk, reference_image_sitk, initial_transform, sitk.sitkLinear, 0.0, img_sitk.GetPixelID())
        resampled_img_nib = nib.Nifti1Image(sitk.GetArrayFromImage(resampled_img_sitk), reference_img_nib.affine)
        resampled_images.append(resampled_img_nib)

    if verbose:
        print("Image resampling completed.")
        print("The shape of the reference image:", reference_img_nib.shape)
        for i, img_nib in enumerate(resampled_images):
            print(f"Resampled Image {i+1} shape: {img_nib.shape}")

    if visualise:
        def visualize_images(images, title_prefix):
            max_slices = min(img.shape[2] for img in images)
            def plot_slices(slice_idx):
                fig, axs = plt.subplots(1, len(images), figsize=(20, 5))
                for i, img in enumerate(images):
                    axs[i].imshow(img.get_fdata()[:, :, slice_idx], cmap="gray")
                    axs[i].axis("off")
                    axs[i].set_title(f"{title_prefix} - Image {i+1}")
                plt.tight_layout()
                plt.show()

            interact(plot_slices, slice_idx=IntSlider(min=0, max=max_slices-1, step=1, value=max_slices//2))

        visualize_images(resampled_images, "After Resampling")

    return resampled_images


def save_resampled_image(image, save_path, file_name):
    """
    Save a resampled NIfTI image to the specified location.

    Args:
        image (nibabel.Nifti1Image): The resampled NIfTI image to save.
        save_path (str): The directory where the image should be saved.
        file_name (str): The name of the file to save, with the correct extension (.nii or .nii.gz).
    """
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)

    if not file_name.endswith(('.nii', '.nii.gz')):
        raise ValueError("File name must have a valid NIfTI extension (.nii or .nii.gz).")

    full_save_path = os.path.join(save_path, file_name)
    try:
        nib.save(image, full_save_path)
        print(f"Image successfully saved to: {full_save_path}")
    except Exception as e:
        print(f"An error occurred while saving the image: {e}")


# Main execution
minboggle_input_folder = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/mindboggle/input"
synaptive_folder = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/synaptive_sample"

# Get file names and paths
mindboggle_files = os.listdir(minboggle_input_folder)
synaptive_files = os.listdir(synaptive_folder)
mindboggle_paths = [os.path.join(minboggle_input_folder, file) for file in mindboggle_files]
synaptive_paths = [os.path.join(synaptive_folder, file) for file in synaptive_files]

# Resample images
resampled_synaptive = adjust_images_to_reference_img(synaptive_paths, verbose=True, visualise=False)

# Save the first resampled image

image_name = "synthseg_0000_0001.nii.gz"

if resampled_synaptive:
    save_resampled_image(
        image=resampled_synaptive[0],
        save_path=path_to_save,
        file_name=image_name
    )
else:
    print("No resampled images to save.")


Image resampling completed.
The shape of the reference image: (240, 240, 155)
Resampled Image 1 shape: (240, 240, 155)
Image successfully saved to: /home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/nnunet/nnUNet_raw/Dataset008_synthseg/imagesTs/synthseg_0000_0001.nii.gz


In [ ]:
import os
import nibabel as nib

minboggle_input_folder = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/data/mindboggle/input"
path_to_save = "/home/fp427/rds/rds-cam-segm-7tts6phZ4tw/mission/nnunet/nnUNet_raw/Dataset008_synthseg/imagesTs"

def copy_mindboggle_files(mindboggle_path, save_path):
    # Get a sorted list of all .nii.gz files in the mindboggle_path
    mindboggle_files = [f for f in os.listdir(mindboggle_path) if f.endswith(".nii.gz")]
    mindboggle_files.sort()

    # Iterate over the files and save them with the new naming convention
    for i, filename in enumerate(mindboggle_files):
        # Construct full input path
        input_file = os.path.join(mindboggle_path, filename)

        # Load the NIfTI file
        img = nib.load(input_file)
        
        # Create the new filename with zero-padded patient_id
        patient_id_str = str(i).zfill(4)
        new_filename = f"synthseg_{patient_id_str}_0001.nii.gz"
        
        # Construct the output path
        output_file = os.path.join(save_path, new_filename)

        # Save the image with the new filename, preserving orientation and affine
        nib.save(img, output_file)


# Run the function
copy_mindboggle_files(minboggle_input_folder, path_to_save)

# check how many files in the imageTs




In [4]:
print(len(os.listdir(path_to_save)))

101
